In [106]:
import csv
import numpy as np
import datetime
import pandas as pd
import datetime
from dateutil import rrule 
import datetime

# Generate ruleset for holiday observances on the NYSE

def NYSE_holidays(a=datetime.date.today(), b=datetime.date.today()+datetime.timedelta(days=365)):
    rs = rrule.rruleset()

    # Include all potential holiday observances
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=31, byweekday=rrule.FR)) # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 1))                     # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 2, byweekday=rrule.MO)) # New Years Day    
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, byweekday= rrule.MO(3)))            # Martin Luther King Day   
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 2, byweekday= rrule.MO(3)))            # Washington's Birthday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, byeaster= -2))                                  # Good Friday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 5, byweekday= rrule.MO(-1)))           # Memorial Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 3, byweekday=rrule.FR)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 4))                     # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 5, byweekday=rrule.MO)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 9, byweekday= rrule.MO(1)))            # Labor Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=11, byweekday= rrule.TH(4)))            # Thanksgiving Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=24, byweekday=rrule.FR)) # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=25))                     # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=26, byweekday=rrule.MO)) # Christmas 
    
    # Exclude potential holidays that fall on weekends
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, until=b, byweekday=(rrule.SA,rrule.SU)))

    return rs
    
# Generate ruleset for NYSE trading days

def NYSE_tradingdays(a=datetime.date.today(), b=datetime.date.today()+datetime.timedelta(days=365)):
    rs = rrule.rruleset()
    rs.rrule(rrule.rrule(rrule.DAILY, dtstart=a, until=b))
    
    # Exclude weekends and holidays
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, byweekday=(rrule.SA,rrule.SU)))
    rs.exrule(NYSE_holidays(a,b))
    
    return rs


# Count NYSE holidays days in next 4 years
hdays =list(NYSE_holidays(datetime.datetime(2011,1,1),datetime.datetime(2014,12,31)))

fields = ['caldt','spindx']
sp500 = pd.read_csv('./Datasets/S&P500.csv', skipinitialspace=True, usecols=fields)
sp500['caldt'] = pd.to_datetime(sp500['caldt'], format='%m/%d/%Y', utc=True)


fields = ['St_date','symbol','Exp_date','Strike_price','Best_bid','Best_offer','Imp_vol','Volume']

#Read options_data
df = pd.read_csv('./Datasets/option_data.csv', skipinitialspace=True, usecols=fields)

#Extract spxw
spxw = df[df['symbol'].str.contains('SPXW')]

#Format Dates
df['St_date'] = pd.to_datetime(df['St_date'], format='%m/%d/%y', utc=True)
df['Exp_date'] = pd.to_datetime(df['Exp_date'], format='%m/%d/%y', utc=True)

#Create Tenor
df.loc[:,'Tenor']=df.loc[:,'Exp_date']-df.loc[:,'St_date']

In [124]:
#Build Moneyness
sLength = len(df['St_date'])
df.loc[:,'Moneyness'] = pd.Series(np.random.randn(sLength), index=df.index)
for row in df.itertuples(index=True, name='Pandas'):
    dt=getattr(row, "St_date")
    K=getattr(row, "Strike_price")
    t=getattr(row, "Tenor")
    F=float(sp500[sp500['caldt']==dt]['spindx'])
    
    #find differences and sort
    interm=df.loc[df['St_date']==dt]
    diff = interm.Strike_price.apply(lambda z: abs(F-z))
    diff.sort_values()
    
    #get the index for the closest number
    inds = diff.index[0]
    s = interm['Imp_vol']
    I=s[inds]
    m=np.log(K/F)/(np.sqrt(t.days)*I)
    df.set_value(row.Index, 'Moneyness', m)
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi




Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi

Hi


KeyboardInterrupt: 

In [61]:
#Delete 0 volume entries
df=df.loc[df['Volume']>0]

#Tenor criterion
datelist = df['St_date'].unique()
for i in range(df['Tenor'].max().days):
    for item in datelist:
        temp=df.loc[(df['Tenor']==datetime.timedelta(0,i)) & (df['St_date']==item)]
        if a<10:
            df=df.loc[(df['Tenor']!=datetime.timedelta(0,i)) | (df['St_date']!=item)]

#Maturity criterion
df=df.loc[df['Exp_date']-df['St_date']>datetime.timedelta(0, 9)]
df=df.loc[df['Exp_date']-df['St_date']<datetime.timedelta(0, 365)]

#Moneyness criterion
df=df.loc[df['Moneyness']>-15]
df=df.loc[df['Moneyness']<5]

#Bid criterion
df=df.loc[df['Best_offer']/df['Best_bid']<5]

#Holiday criterion
for h in hdays:
    df=df.loc[(df['St_date']!=h)]


,St_date,symbol,Exp_date,Strike_price,Best_bid,Best_offer,Volume,Imp_vol,Tenor
940,2011-01-03 00:00:00+00:00,SPXW 110107P1125000,2011-01-07 00:00:00+00:00,1125000,0.05,0.10,416,0.514714,4 days
941,2011-01-03 00:00:00+00:00,SPXW 110107P1150000,2011-01-07 00:00:00+00:00,1150000,0.10,0.15,1270,0.456901,4 days
942,2011-01-03 00:00:00+00:00,SPXW 110107P1175000,2011-01-07 00:00:00+00:00,1175000,0.15,0.20,2666,0.385720,4 days
943,2011-01-03 00:00:00+00:00,SPXW 110107P1200000,2011-01-07 00:00:00+00:00,1200000,0.30,0.35,4495,0.323416,4 days
944,2011-01-03 00:00:00+00:00,SPXW 110107P1205000,2011-01-07 00:00:00+00:00,1205000,0.35,0.40,137,0.310991,4 days
945,2011-01-03 00:00:00+00:00,SPXW 110107P1210000,2011-01-07 00:00:00+00:00,1210000,0.40,0.45,292,0.297236,4 days
946,2011-01-03 00:00:00+00:00,SPXW 110107P1215000,2011-01-07 00:00:00+00:00,1215000,0.45,0.50,732,0.282313,4 days
947,2011-01-03 00:00:00+00:00,SPXW 110107P1220000,2011-01-07 00:00:00+00:00,1220000,0.50,0.60,192,0.268724,4 days
948,2011-01-03 00:00:00+00:00,SPXW 110107P1225000,2011-01-07 00:00:00+00:00,1225000,0.50,0.65,1322,0.249335,4 days
949,2011-01-03 00:00:00+00:00,SPXW 110107P1230000,2011-01-07 00:00:00+00:00,1230000,0.60,0.80,795,0.237065,4 days


In [79]:
#Separate short dated
short_dated=df.loc[df['Exp_date']-df['St_date']<datetime.timedelta(0, 9)]
long_dated=df.loc[df['Exp_date']-df['St_date']<datetime.timedelta(0, 9)]

In [90]:
sp500

,caldt,spindx
0,2011-01-03 00:00:00+00:00,1271.87
1,2011-01-04 00:00:00+00:00,1270.20
2,2011-01-05 00:00:00+00:00,1276.56
3,2011-01-06 00:00:00+00:00,1273.85
4,2011-01-07 00:00:00+00:00,1271.50
5,2011-01-10 00:00:00+00:00,1269.75
6,2011-01-11 00:00:00+00:00,1274.48
7,2011-01-12 00:00:00+00:00,1285.96
8,2011-01-13 00:00:00+00:00,1283.76
9,2011-01-14 00:00:00+00:00,1293.24


In [104]:
float(sp500[sp500['spindx']==1271.87]['spindx'])

1271.87

In [111]:
np.log(3.4)

1.2237754316221157

In [121]:
print(datetime.timedelta(0,1))

0:00:01
